In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math

# Debugging of the Serre code

The plan is:

* Use the cnoidal periodic solution of the Serre equations to test the implementation of the different types of Serre models we have
* Start from the top interface and dig into the details

## Second order model with Strang splitting 

In [4]:
import nswe
import cnoidal
import serre

AttributeError: 'module' object has no attribute 'fluxes2'

In [ ]:
import nswe
import cnoidal
import serre
import muscl2

### Cnoidal wave parameters

In [ ]:
a0 = 0.3
a1 = 0.1
k = 0.99
g = 9.81
t = 0.0

lam, h0 = cnoidal.WaveLengthDepth(k,a0,a1)

### Domain discretization

In [ ]:
L = 2*lam
xmin = -L
xmax = L
nx = 100
dx1 = (xmax-xmin)/(nx)
x,dx = serre.discretizeSpace(xmin-2.*dx1,xmax+2.*dx1,nx+4)

### Initial condition

Notice that this definition does not include ghost-cells, this is to avoid confusion with periodic boundaries and ghost cells.


In [ ]:
h,u = cnoidal.analyticalSolution(x[2:-2],t,k,a0,a1)
plt.figure(figsize=(6,2))
plt.subplot(121)
plt.plot(x[2:-2],h)
plt.ylabel('h')
plt.xlabel('x')
plt.subplot(122)
plt.plot(x[2:-2],u)
plt.ylabel('u')
plt.xlabel('x')
plt.tight_layout()

### Boundary conditions
Now extend the domain to periodic boundaries with 2 ghost cells on each side.

In [ ]:
h = serre.extend2GhostCells(h,2)
u = serre.extend2GhostCells(u,2)
h,u = serre.periodicDomainTwoGC(h,u,None,dx,0)

## Simulation using three steps in the Splitting $\left(S_1(\Delta t) S_2(\Delta t) S_1(\Delta t)\right)$

In [ ]:
hall1,uall1,tall1 = serre.splitSerre(x,h,u,0,10.,serre.periodicDomainTwoGC,
                                     serre.periodicDomain2TwoGC,None,None,
                                     dx,nx,vardt = False, dt = 0.05, splitSteps=3,
                                     order=2, periodic=True,
                                     fvsolver = muscl2.fluxes2, fdsolver = serre.EFDSolverFM, ghostcells=2)

## Simulation with the second order non linear part only

In [ ]:
u2 = np.copy(u)
h2 = np.copy(h)
hall2,uall2,tall2 = serre.NSWE(x,h2,u2,0,10.,serre.periodicDomainTwoGC,None,dx,nx,vardt = False, dt = 0.05,
                              fvsolver=muscl2.fluxes2, ghostcells=2)

## Simulation with first order  $S_1$ and second order $S_2$, using three steps in the splitting

In [ ]:
nx = nx
dx1 = (xmax-xmin)/(nx)

print(x.shape)
x,dx = serre.discretizeSpace(xmin-1.*dx1,xmax+1.*dx1,nx+2)
print(x.shape)
h,u = cnoidal.analyticalSolution(x[1:-1],t,k,a0,a1)
h = serre.extend2GhostCells(h,1)
u = serre.extend2GhostCells(u,1)
h,u = serre.periodicDomain(h,u,None,dx,0)

hall3,uall3,tall3 = serre.splitSerre(x,h,u,0,10.,serre.periodicDomain,
                                     serre.periodicDomain2,None,None,dx,nx+2,vardt = False, dt = 0.05, splitSteps=3,
                                     order=2, periodic=True,
                                     fvsolver = nswe.fluxes, fdsolver = serre.EFDSolverFM, ghostcells=1)

h = serre.extend2GhostCells(h,1)
u = serre.extend2GhostCells(u,1)
x = serre.extend2GhostCells(x,1)
x[0] = x[1]-dx
x[-1] = x[-2]+dx

umin = np.amin(np.concatenate((uall1,uall2,uall3)))
umax = np.amax(np.concatenate((uall1,uall2,uall3)))
hmin = np.amin(np.concatenate((hall1,hall2,hall3)))
hmax = np.amax(np.concatenate((hall1,hall2,hall3)))

print(np.linalg.norm(uall1-uall2))
print(np.linalg.norm(hall1-hall2))

## Compute analytical solution

In [ ]:
hallexact = np.copy(hall1)
uallexact = np.copy(uall1)

for i in range (tall1.size):
    hallexact[2:-2,i],uallexact[2:-2,i] = cnoidal.analyticalSolution(x,tall1[i],k,a0,a1)

## Plot everything to compare

In [ ]:
import generalFunctions as gF

In [ ]:
gF.plotAnimationNSolutions?

In [ ]:
print hall1.shape,hall2.shape,hall3.shape,hallexact.shape

In [ ]:
lb = ["Serre","NSWE","Serre FV Order 1", "Analytical solution"]
gF.plotAnimationNSolutions(4,x[2:-2],np.array([hall1[2:-2],hall2[2:-2],hall3,hallexact[2:-2]]),tall1,xmin,xmax,hmin,hmax,lb,ylabel=r'$h$')